In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
import scipy.stats as stats
from scipy.stats import skew
import statsmodels.api as sm

%matplotlib inline

In [2]:
ACCRE_df = pd.read_csv('data/fullsample.csv')

In [3]:
ACCRE_df.head()

,JOBID,ACCOUNT,USER,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE,SUBMIT
0,6794220,laxtons,mikala,247808Mn,115217.12M,14-00:00:00,11-01:32:50,1,16,production,0:0,2019-03-09T16:38:44
1,6983193_1880,discovery,serina,12288Mn,6516.10M,13-18:00:00,12-09:26:41,1,1,production,0:0,2019-03-07T18:39:59
2,7055502_1863,discovery,serina,12288Mn,0.09M,13-18:00:00,13-08:27:36,1,1,production,0:0,2019-03-11T10:06:06
3,7055502_1800,discovery,serina,12288Mn,8333.82M,13-18:00:00,12-03:50:59,1,1,production,0:0,2019-03-11T10:06:06
4,7055502_1806,discovery,serina,12288Mn,8304.03M,13-18:00:00,11-08:43:46,1,1,production,0:0,2019-03-11T10:06:06


In [4]:
ACCRE_df.info()
# Notice 4,365,379 entries

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4365379 entries, 0 to 4365378
Data columns (total 12 columns):
JOBID        object
ACCOUNT      object
USER         object
REQMEM       object
USEDMEM      object
REQTIME      object
USEDTIME     object
NODES        int64
CPUS         int64
PARTITION    object
EXITCODE     object
SUBMIT       object
dtypes: int64(2), object(10)
memory usage: 399.7+ MB


In [5]:
ACCRE_df.tail()
# No footer to remove

,JOBID,ACCOUNT,USER,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE,SUBMIT
4365374,12741062_145,mickey,boyce,4096Mc,462.90M,00:10:00,00:01:44,1,1,production,0:0,2019-09-25T14:22:03
4365375,12741062_146,mickey,boyce,4096Mc,462.89M,00:10:00,00:01:44,1,1,production,0:0,2019-09-25T14:22:03
4365376,12741062_147,mickey,boyce,4096Mc,553.77M,00:10:00,00:01:53,1,1,production,0:0,2019-09-25T14:22:03
4365377,12741062_148,mickey,boyce,4096Mc,557.09M,00:10:00,00:01:52,1,1,production,0:0,2019-09-25T14:22:03
4365378,12741062_149,mickey,boyce,4096Mc,616.77M,00:10:00,00:01:52,1,1,production,0:0,2019-09-25T14:22:03


In [6]:
ACCRE_df.head()

,JOBID,ACCOUNT,USER,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE,SUBMIT
0,6794220,laxtons,mikala,247808Mn,115217.12M,14-00:00:00,11-01:32:50,1,16,production,0:0,2019-03-09T16:38:44
1,6983193_1880,discovery,serina,12288Mn,6516.10M,13-18:00:00,12-09:26:41,1,1,production,0:0,2019-03-07T18:39:59
2,7055502_1863,discovery,serina,12288Mn,0.09M,13-18:00:00,13-08:27:36,1,1,production,0:0,2019-03-11T10:06:06
3,7055502_1800,discovery,serina,12288Mn,8333.82M,13-18:00:00,12-03:50:59,1,1,production,0:0,2019-03-11T10:06:06
4,7055502_1806,discovery,serina,12288Mn,8304.03M,13-18:00:00,11-08:43:46,1,1,production,0:0,2019-03-11T10:06:06


In [7]:
ACCRE_df.dropna().head()
# This can be ignored because drop.duplicates is coming next

,JOBID,ACCOUNT,USER,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE,SUBMIT
0,6794220,laxtons,mikala,247808Mn,115217.12M,14-00:00:00,11-01:32:50,1,16,production,0:0,2019-03-09T16:38:44
1,6983193_1880,discovery,serina,12288Mn,6516.10M,13-18:00:00,12-09:26:41,1,1,production,0:0,2019-03-07T18:39:59
2,7055502_1863,discovery,serina,12288Mn,0.09M,13-18:00:00,13-08:27:36,1,1,production,0:0,2019-03-11T10:06:06
3,7055502_1800,discovery,serina,12288Mn,8333.82M,13-18:00:00,12-03:50:59,1,1,production,0:0,2019-03-11T10:06:06
4,7055502_1806,discovery,serina,12288Mn,8304.03M,13-18:00:00,11-08:43:46,1,1,production,0:0,2019-03-11T10:06:06


In [8]:
ACCRE_df = ACCRE_df.drop_duplicates()

In [9]:
ACCRE_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3450799 entries, 0 to 4365378
Data columns (total 12 columns):
JOBID        object
ACCOUNT      object
USER         object
REQMEM       object
USEDMEM      object
REQTIME      object
USEDTIME     object
NODES        int64
CPUS         int64
PARTITION    object
EXITCODE     object
SUBMIT       object
dtypes: int64(2), object(10)
memory usage: 342.3+ MB


In [10]:
ACCRE_df_abridged = pd.read_csv('data/fullsample_dedup.csv')
# Reading in this csv is just to check if we eliminated the correct number of entries. 
# The ACCRE guy gave us this edited csv. 

In [11]:
ACCRE_df_abridged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3450799 entries, 0 to 3450798
Data columns (total 12 columns):
JOBID        object
ACCOUNT      object
USER         object
REQMEM       object
USEDMEM      object
REQTIME      object
USEDTIME     object
NODES        int64
CPUS         int64
PARTITION    object
EXITCODE     object
SUBMIT       object
dtypes: int64(2), object(10)
memory usage: 315.9+ MB


In [12]:
ACCRE_df = ACCRE_df.loc[ACCRE_df['PARTITION'] == 'production']
ACCRE_df.head()
# We want to ignore the partition named "debug"

,JOBID,ACCOUNT,USER,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE,SUBMIT
0,6794220,laxtons,mikala,247808Mn,115217.12M,14-00:00:00,11-01:32:50,1,16,production,0:0,2019-03-09T16:38:44
1,6983193_1880,discovery,serina,12288Mn,6516.10M,13-18:00:00,12-09:26:41,1,1,production,0:0,2019-03-07T18:39:59
2,7055502_1863,discovery,serina,12288Mn,0.09M,13-18:00:00,13-08:27:36,1,1,production,0:0,2019-03-11T10:06:06
3,7055502_1800,discovery,serina,12288Mn,8333.82M,13-18:00:00,12-03:50:59,1,1,production,0:0,2019-03-11T10:06:06
4,7055502_1806,discovery,serina,12288Mn,8304.03M,13-18:00:00,11-08:43:46,1,1,production,0:0,2019-03-11T10:06:06


In [13]:
ACCRE_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3319257 entries, 0 to 4365378
Data columns (total 12 columns):
JOBID        object
ACCOUNT      object
USER         object
REQMEM       object
USEDMEM      object
REQTIME      object
USEDTIME     object
NODES        int64
CPUS         int64
PARTITION    object
EXITCODE     object
SUBMIT       object
dtypes: int64(2), object(10)
memory usage: 329.2+ MB


In [14]:
ACCRE_df.REQTIME = ACCRE_df.REQTIME.str.replace('-',':')
ACCRE_df.USEDTIME = ACCRE_df.USEDTIME.str.replace('-',':')
# REQTIME and USEDTIME are now manageable 

In [15]:
ACCRE_df.head()

,JOBID,ACCOUNT,USER,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE,SUBMIT
0,6794220,laxtons,mikala,247808Mn,115217.12M,14:00:00:00,11:01:32:50,1,16,production,0:0,2019-03-09T16:38:44
1,6983193_1880,discovery,serina,12288Mn,6516.10M,13:18:00:00,12:09:26:41,1,1,production,0:0,2019-03-07T18:39:59
2,7055502_1863,discovery,serina,12288Mn,0.09M,13:18:00:00,13:08:27:36,1,1,production,0:0,2019-03-11T10:06:06
3,7055502_1800,discovery,serina,12288Mn,8333.82M,13:18:00:00,12:03:50:59,1,1,production,0:0,2019-03-11T10:06:06
4,7055502_1806,discovery,serina,12288Mn,8304.03M,13:18:00:00,11:08:43:46,1,1,production,0:0,2019-03-11T10:06:06


In [16]:
date_time=ACCRE_df.REQTIME

In [17]:
%%time
# REQTIME needs to be in total seconds.
to_seconds = []
for time in date_time:
    split=time.split(':')
    if len(split)==4:
        seconds=int(split[0])*86400+int(split[1])*3600+int(split[2])*60+int(split[3])
        to_seconds.append(seconds)
    else: 
        seconds=int(split[0])*3600+int(split[1])*60+int(split[2])
        to_seconds.append(seconds)

CPU times: user 5.65 s, sys: 79.1 ms, total: 5.73 s
Wall time: 5.77 s


In [18]:
ACCRE_df.REQTIME = to_seconds
ACCRE_df.head()

,JOBID,ACCOUNT,USER,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE,SUBMIT
0,6794220,laxtons,mikala,247808Mn,115217.12M,1209600,11:01:32:50,1,16,production,0:0,2019-03-09T16:38:44
1,6983193_1880,discovery,serina,12288Mn,6516.10M,1188000,12:09:26:41,1,1,production,0:0,2019-03-07T18:39:59
2,7055502_1863,discovery,serina,12288Mn,0.09M,1188000,13:08:27:36,1,1,production,0:0,2019-03-11T10:06:06
3,7055502_1800,discovery,serina,12288Mn,8333.82M,1188000,12:03:50:59,1,1,production,0:0,2019-03-11T10:06:06
4,7055502_1806,discovery,serina,12288Mn,8304.03M,1188000,11:08:43:46,1,1,production,0:0,2019-03-11T10:06:06


In [19]:
used_time = ACCRE_df.USEDTIME

In [20]:
def convert_to_seconds(new):
    split_again=new.split(':')
    if len(split_again)==4: 
        sec_2=int(split_again[0])*86400+int(split_again[1])*3600+int(split_again[2])*60+int(split_again[3])
    
    else: 
        sec_2=int(split_again[0])*3600+int(split_again[1])*60+int(split_again[2])
               
    return sec_2

In [21]:
%%time 
ACCRE_df.USEDTIME = ACCRE_df.USEDTIME.apply(convert_to_seconds)

CPU times: user 5.69 s, sys: 190 ms, total: 5.88 s
Wall time: 5.72 s


In [22]:
ACCRE_df.head()

,JOBID,ACCOUNT,USER,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE,SUBMIT
0,6794220,laxtons,mikala,247808Mn,115217.12M,1209600,955970,1,16,production,0:0,2019-03-09T16:38:44
1,6983193_1880,discovery,serina,12288Mn,6516.10M,1188000,1070801,1,1,production,0:0,2019-03-07T18:39:59
2,7055502_1863,discovery,serina,12288Mn,0.09M,1188000,1153656,1,1,production,0:0,2019-03-11T10:06:06
3,7055502_1800,discovery,serina,12288Mn,8333.82M,1188000,1050659,1,1,production,0:0,2019-03-11T10:06:06
4,7055502_1806,discovery,serina,12288Mn,8304.03M,1188000,981826,1,1,production,0:0,2019-03-11T10:06:06


In [23]:
head=ACCRE_df.head()
# Just want to mess with a (very small) subset

In [24]:
#head.REQMEM = head['REQMEM'].str.replace(r'(Mn|Mc)', r' \1')
#head
# We need a space between values and units so we can split 

In [25]:
#head.USEDMEM = head['USEDMEM'].str.replace(r'(M)', r' \1')
#head

In [26]:
#iterrows, itertuples?

In [27]:
def convert_to_mc(x,y,z): 
    num_unit=x.split(' ')
    number=int(num_unit[0])
    unit=num_unit[1]
    if unit =='Mn': 
        mc_conversion=(number*y)/(z) 
        
    else: 
        mc_conversion=number 
    
    return mc_conversion

In [28]:
#head.info()

In [29]:
#%%time
#head['REQMEM']=head.apply(lambda row: convert_to_mc(row['REQMEM'],row['NODES'],row['CPUS']),axis=1)

In [30]:
#head

In [31]:
def m_to_mc(a,b,c): 
    if a =='0':
        mc=a 
    else: 
        split_it=a.split(' ')
        value=float(split_it[0])
        letter=split_it[1]
        mc=(value*b)/(c)
    
    
    return mc

In [32]:
#%%time
#head['USEDMEM']=head.apply(lambda row: m_to_mc(row['USEDMEM'],row['NODES'],row['CPUS']),axis=1)

In [33]:
#head=m_to_mc(head)
#head

In [34]:
ACCRE_df.head()

,JOBID,ACCOUNT,USER,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE,SUBMIT
0,6794220,laxtons,mikala,247808Mn,115217.12M,1209600,955970,1,16,production,0:0,2019-03-09T16:38:44
1,6983193_1880,discovery,serina,12288Mn,6516.10M,1188000,1070801,1,1,production,0:0,2019-03-07T18:39:59
2,7055502_1863,discovery,serina,12288Mn,0.09M,1188000,1153656,1,1,production,0:0,2019-03-11T10:06:06
3,7055502_1800,discovery,serina,12288Mn,8333.82M,1188000,1050659,1,1,production,0:0,2019-03-11T10:06:06
4,7055502_1806,discovery,serina,12288Mn,8304.03M,1188000,981826,1,1,production,0:0,2019-03-11T10:06:06


In [35]:
ACCRE_df.tail()

,JOBID,ACCOUNT,USER,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE,SUBMIT
4365374,12741062_145,mickey,boyce,4096Mc,462.90M,600,104,1,1,production,0:0,2019-09-25T14:22:03
4365375,12741062_146,mickey,boyce,4096Mc,462.89M,600,104,1,1,production,0:0,2019-09-25T14:22:03
4365376,12741062_147,mickey,boyce,4096Mc,553.77M,600,113,1,1,production,0:0,2019-09-25T14:22:03
4365377,12741062_148,mickey,boyce,4096Mc,557.09M,600,112,1,1,production,0:0,2019-09-25T14:22:03
4365378,12741062_149,mickey,boyce,4096Mc,616.77M,600,112,1,1,production,0:0,2019-09-25T14:22:03


In [36]:
ACCRE_df.REQMEM = ACCRE_df['REQMEM'].str.replace(r'(Mn|Mc)', r' \1')
ACCRE_df.USEDMEM = ACCRE_df['USEDMEM'].str.replace(r'(M)', r' \1')
ACCRE_df.head()

,JOBID,ACCOUNT,USER,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE,SUBMIT
0,6794220,laxtons,mikala,247808 Mn,115217.12 M,1209600,955970,1,16,production,0:0,2019-03-09T16:38:44
1,6983193_1880,discovery,serina,12288 Mn,6516.10 M,1188000,1070801,1,1,production,0:0,2019-03-07T18:39:59
2,7055502_1863,discovery,serina,12288 Mn,0.09 M,1188000,1153656,1,1,production,0:0,2019-03-11T10:06:06
3,7055502_1800,discovery,serina,12288 Mn,8333.82 M,1188000,1050659,1,1,production,0:0,2019-03-11T10:06:06
4,7055502_1806,discovery,serina,12288 Mn,8304.03 M,1188000,981826,1,1,production,0:0,2019-03-11T10:06:06


In [37]:
%%time
ACCRE_df['REQMEM']=ACCRE_df.apply(lambda row: convert_to_mc(row['REQMEM'],row['NODES'],row['CPUS']),axis=1)

CPU times: user 1min 58s, sys: 2.94 s, total: 2min 1s
Wall time: 2min 6s


In [38]:
ACCRE_df.head()

,JOBID,ACCOUNT,USER,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE,SUBMIT
0,6794220,laxtons,mikala,15488.0,115217.12 M,1209600,955970,1,16,production,0:0,2019-03-09T16:38:44
1,6983193_1880,discovery,serina,12288.0,6516.10 M,1188000,1070801,1,1,production,0:0,2019-03-07T18:39:59
2,7055502_1863,discovery,serina,12288.0,0.09 M,1188000,1153656,1,1,production,0:0,2019-03-11T10:06:06
3,7055502_1800,discovery,serina,12288.0,8333.82 M,1188000,1050659,1,1,production,0:0,2019-03-11T10:06:06
4,7055502_1806,discovery,serina,12288.0,8304.03 M,1188000,981826,1,1,production,0:0,2019-03-11T10:06:06


In [48]:
ACCRE_df.loc[ACCRE_df['CPUS']==0]

,JOBID,ACCOUNT,USER,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE,SUBMIT
1556155,9524253,kumquat,kate,16384.0,16383.34 M,86400,35114,24,0,production,0:0,2019-07-01T14:22:00


In [50]:
ACCRE_df.drop(ACCRE_df[ACCRE_df['CPUS']==0].index, inplace=True)

In [51]:
%%time
ACCRE_df['USEDMEM']=ACCRE_df.apply(lambda row: m_to_mc(row['USEDMEM'],row['NODES'],row['CPUS']),axis=1)

CPU times: user 2min 1s, sys: 2.17 s, total: 2min 3s
Wall time: 2min 7s


In [40]:
# Recall that you have made this function: 
#def convert_to_mc(df):
    #num_unit=df.REQMEM.str.split(' ')
    
    #df.REQMEM = num_unit.map(lambda x: int(x[0]))
    #df['unit'] = num_unit.map(lambda x: x[1])
    #idx = df.unit=='Mn'
    #df.REQMEM.loc[idx]=df.REQMEM*df.NODES/df.CPUS
    
    #return df

In [41]:
# def convert_to_mc(df):
   # num_unit=df.REQMEM.str.split(' ')
    
   # df.REQMEM = num_unit.map(lambda x: int(x[0])) # changed to float, was int
   # df['unit'] = num_unit.map(lambda x: x[1]) # made float
   # idx = df.unit=='Mn|Mc'
   # df.REQMEM.loc[idx]=(df.REQMEM*df.NODES)/df.CPUS
    
   # return df

In [42]:
# def convert_to_mc_M(df):
  #  num_unit_M=df.USEDMEM.str.split(' ')
  #  df.USEDMEM = num_unit_M.map(lambda x: float(x[0]))
  #  df['unit_M'] = num_unit_M.map(lambda x: x[1])
  #  idx = df.unit_M=='M'
  #  df.USEDMEM.loc[idx]=df.USEDMEM*df.NODES/df.CPUS
    
  #  return df

In [43]:
ACCRE_df.groupby('ACCOUNT').mean()

,REQMEM,REQTIME,USEDTIME,NODES,CPUS
ACCOUNT,,,,,
alkmene,11553.681629,140727.352586,21832.211412,1.000000,1.212962
almonds,4593.089043,67623.508907,21111.612062,1.000000,1.024097
amaranth,9409.086170,23488.607595,1323.247539,1.000000,4.326301
antares,6040.981151,170744.639463,47920.031604,1.000000,2.664813
arra,10945.168061,60430.722433,3005.111787,1.044867,2.872243
...,...,...,...,...,...
virginia,2280.244099,188084.974093,72176.413356,1.000000,1.000000
wax,8990.063456,30932.385262,10418.245637,1.000323,3.724952
white,4915.200000,43200.000000,2319.980000,1.000000,1.000000


In [44]:
ACCRE_df.groupby('ACCOUNT').describe()

REQMEM                                                            \
             count          mean          std          min      25%      50%   
ACCOUNT                                                                        
alkmene    81963.0  11553.681629  4486.233940  2000.000000   8192.0  11000.0   
almonds    88643.0   4593.089043  2875.242143  1000.000000   4096.0   4096.0   
amaranth     711.0   9409.086170  3783.354127  1024.000000   8192.0  12288.0   
antares   178900.0   6040.981151  1176.139476  1024.000000   5000.0   7000.0   
arra        1315.0  10945.168061  7447.522478  1536.000000   8000.0   8000.0   
...            ...           ...          ...          ...      ...      ...   
virginia    1737.0   2280.244099   659.369239   500.000000   2048.0   2048.0   
wax         3094.0   8990.063456  6224.794916   682.666667   4096.0   5120.0   
white        100.0   4915.200000  1646.653957  4096.000000   4096.0   4096.0   
winged     15996.0  15075.017254  6473.414246  1000.000000  12288.0  20000.0   
with           1.0   1800.000000          NaN  1800.000000   1800.0   1800.0   

                             REQTIME                 ... NODES           CPUS  \
              75%      max     count           mean  ...   75%  max     count   
ACCOUNT                                              ...                        
alkmene   16384.0  20000.0   81963.0  140727.352586  ...   1.0  1.0   81963.0   
almonds    4500.0  20480.0   88643.0   67623.508907  ...   1.0  1.0   88643.0   
amaranth  12288.0  19660.8     711.0   23488.607595  ...   1.0  1.0     711.0   
antares    7000.0  10240.0  178900.0  170744.639463  ...   1.0  1.0  178900.0   
arra      12500.0  40000.0    1315.0   60430.722433  ...   1.0  2.0    1315.0   
...           ...      ...       ...            ...  ...   ...  ...       ...   
virginia   2048.0   4096.0    1737.0  188084.974093  ...   1.0  1.0    1737.0   
wax       16384.0  32768.0    3094.0   30932.385262  ...   1.0  2.0    3094.0   
white      4096.0   8192.0     100.0   43200.000000  ...   1.0  1.0     100.0   
winged    20480.0  20480.0   15996.0   95861.785446  ...   1.0  1.0   15996.0   
with       1800.0   1800.0       1.0  259200.000000  ...   1.0  1.0       1.0   

                                                             
               mean       std   min   25%   50%   75%   max  
ACCOUNT                                                      
alkmene    1.212962  0.549004   1.0   1.0   1.0   1.0   5.0  
almonds    1.024097  0.154887   1.0   1.0   1.0   1.0   4.0  
amaranth   4.326301  3.242825   1.0   1.0   2.0   8.0  16.0  
antares    2.664813  1.491132   1.0   1.0   4.0   4.0   8.0  
arra       2.872243  2.800106   1.0   1.0   1.0   5.0   8.0  
...             ...       ...   ...   ...   ...   ...   ...  
virginia   1.000000  0.000000   1.0   1.0   1.0   1.0   1.0  
wax        3.724952  3.006456   1.0   1.0   4.0   4.0  16.0  
white      1.000000  0.000000   1.0   1.0   1.0   1.0   1.0  
winged     1.234996  0.607633   1.0   1.0   1.0   1.0   8.0  
with      10.000000       NaN  10.0  10.0  10.0  10.0  10.0  

[115 rows x 40 columns]

In [45]:
ACCRE_df.groupby('ACCOUNT').diff().describe()

,REQMEM,REQTIME,USEDTIME,NODES,CPUS
count,3.319142e+06,3.319142e+06,3.319142e+06,3.319142e+06,3.319142e+06
mean,2.113932e-01,-9.633755e-01,-1.722592e+00,4.790395e-05,-1.205131e-06
std,2.778584e+03,4.030782e+04,2.249300e+04,3.263713e-01,5.633095e-01
min,-1.280000e+06,-1.208400e+06,-1.197053e+06,-9.900000e+01,-2.300000e+01
25%,0.000000e+00,0.000000e+00,-1.060000e+02,0.000000e+00,0.000000e+00
50%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,0.000000e+00,0.000000e+00,1.040000e+02,0.000000e+00,0.000000e+00
max,2.044928e+06,1.208400e+06,1.192082e+06,1.900000e+02,2.300000e+01


In [46]:
ACCRE_df.groupby('ACCOUNT').diff(axis=1)

""


In [54]:
ACCRE_df['MEM_diff'] = ACCRE_df['REQMEM'].sub(ACCRE_df['USEDMEM'], axis = 0)

In [52]:
ACCRE_df['REQMEM'] = ACCRE_df['REQMEM'].astype('int64')

In [53]:
ACCRE_df['USEDMEM'] = ACCRE_df['USEDMEM'].astype('int64')

In [56]:
ACCRE_df.head()

,JOBID,ACCOUNT,USER,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE,SUBMIT,MEM_diff
0,6794220,laxtons,mikala,15488,7201,1209600,955970,1,16,production,0:0,2019-03-09T16:38:44,8287
1,6983193_1880,discovery,serina,12288,6516,1188000,1070801,1,1,production,0:0,2019-03-07T18:39:59,5772
2,7055502_1863,discovery,serina,12288,0,1188000,1153656,1,1,production,0:0,2019-03-11T10:06:06,12288
3,7055502_1800,discovery,serina,12288,8333,1188000,1050659,1,1,production,0:0,2019-03-11T10:06:06,3955
4,7055502_1806,discovery,serina,12288,8304,1188000,981826,1,1,production,0:0,2019-03-11T10:06:06,3984
